## Transformada de Fourier para datos muestreados no uniformemente

Cuando una señal no está muestreada uniformemente, la **Transformada Discreta de Fourier (DFT)** estándar no puede aplicarse directamente. Estos son los métodos matemáticos para manejar tales casos:

### 1. Interpolación + FFT (Aproximación)

Primero interpolamos los datos no uniformes a una rejilla uniforme:

$
\tilde{x}(t_k) \approx x(t) \quad \text{para} \quad t_k = k\Delta t
$

Luego aplicamos la FFT estándar.



In [2]:
from scipy.interpolate import interp1d
import numpy as np

# Datos no uniformes (ej: tiempos t y valores x)
t_non_uniform = np.array([0, 1.1, 2.5, 3.0, 5.2])
x_non_uniform = np.array([1.0, 2.0, 1.5, 0.5, 1.0])

# Interpolación a una rejilla uniforme
t_uniform = np.linspace(t_non_uniform.min(), t_non_uniform.max(), 100)
interp_func = interp1d(t_non_uniform, x_non_uniform, kind='cubic')
x_uniform = interp_func(t_uniform)

# FFT estándar
X = np.fft.fft(x_uniform)

### 2. Transformada de Fourier Discreta No Uniforme (NUDFT)

Generalización de la DFT para tiempos de muestreo no uniformes $t_n$:

#### a) NUDFT Tipo 1 (Directa)

$
X(f_k) = \sum_{n=0}^{N-1} x(t_n) e^{-j2\pi f_k t_n}
$

#### b) NUDFT Tipo 2 (Inversa)

$
x(t_n) = \sum_{k=0}^{N-1} X(f_k) e^{j2\pi f_k t_n}
$




In [ ]:
from pynfft import NFFT

# Datos no uniformes
t_non_uniform = np.array([0, 0.5, 1.9, 2.1])
x_non_uniform = np.array([1.0, 2.0, 0.5, 1.5])



N = len(t_non_uniform)
freqs = np.fft.fftfreq(N)
plan = NFFT(N, N)
plan.x = t_non_uniform  # Tiempos no uniformes
plan.precompute()
plan.f = x_non_uniform
X_nudft = plan.adjoint()

### 3. Periodograma Lomb-Scargle

Método espectral para datos altamente irregulares:

$
P(f) = \frac{1}{2} \left( \frac{ \left| \sum_n x_n \cos(2\pi f [t_n - \tau]) \right|^2 }{ \sum_n \cos^2(2\pi f [t_n - \tau]) } + \frac{ \left| \sum_n x_n \sin(2\pi f [t_n - \tau]) \right|^2 }{ \sum_n \sin^2(2\pi f [t_n - \tau]) } \right)
$

donde $\tau$ es un desplazamiento temporal que asegura invariancia a traslaciones.

In [ ]:
from astropy.timeseries import LombScargle

# Datos no uniformes
t_non_uniform = np.array([0, 1.1, 2.5, 3.0, 5.2])
x_non_uniform = np.array([1.0, 2.0, 1.5, 0.5, 1.0])

# Periodograma Lomb-Scargle
frequency = np.linspace(0.1, 10, 1000)
power = LombScargle(t_non_uniform, x_non_uniform).power(frequency)

# Gráfica
plt.plot(frequency, power)
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Potencia")
plt.title("Periodograma Lomb-Scargle")
plt.show()

### Comparación de métodos

| Método               | Ventajas                          | Desventajas                     |
|----------------------|-----------------------------------|----------------------------------|
| Interpolación + FFT  | Rápido y simple                  | Introduce error por interpolación|
| NUDFT                | Exacto para muestreo no uniforme | Alto costo computacional        |
| Lomb-Scargle         | Ideal para datos muy irregulares | Solo estima potencia (no fase)  |

### Conclusión

La elección del método depende de:
1. La regularidad del muestreo temporal
2. Los requisitos de precisión
3. Los recursos computacionales disponibles